In [17]:
import numpy as np
import pandas as pd
from linearmodels import PanelOLS
import statsmodels.api as sm
from scipy import stats

In [18]:
df_New_Main = pd.read_excel('TAMU_NewForm_Main_20170817.xlsx')
df_Old_Main = pd.read_excel('TAMU_OldForm_Main_20170817.xlsx')
df_PCG_Data = pd.read_csv('PCG_Donor_Data2.csv')

In [19]:
#do all renaming here
df_PCG_All = df_PCG_Data.rename(index=str, columns = {"Unnamed: 0" : "org_name"})
df_Old_All = df_Old_Main.rename(index=str, columns = {"ein" : "EIN"})[['org_name', 'EIN', 'FiscalYearEnd',
                                                      'NTEECODE', 'government_grants', 
                                                      'service_revenue', 'membership_dues', 'fundraising']]
df_New_All = df_New_Main.rename(index=str, columns = {"ORGNAME" : "org_name"})[['org_name', 'EIN', 'FiscalYearEnd', 
                                                      'NTEECODE', 'FED_CAMPAIGNS', 'MEMBERSHIP_DUES', 
                                                      'TOTAL_FUNDRAISING', 'RELATED_ORGS', 'GOV_GRANTS', 'CONTRIB_OTHER', 
                                                      'CONTRIB_NONCASH', 'CONTRIB_ALL', 'SERVICE_REVENUE']].rename(index=str, columns = {"MEMBERSHIP_DUES" : "membership_dues", 
                                                                           "GOV_GRANTS" : "government_grants", 
                                                                           "SERVICE_REVENUE" : "service_revenue", "TOTAL_FUNDRAISING" : "fundraising"})

In [20]:
#aggregate summing of repeated EINs
df_PCG_All.loc[df_PCG_All['EIN'] == '23-7394620', 'org_name'] = 'University of Alaska'
df_PCG_All.loc[df_PCG_All['EIN'] == '92-0155067', 'org_name'] = 'Alaska Community Foundation'
df_PCG_All.loc[df_PCG_All['EIN'] == '92-0152088', 'org_name'] = 'Alaska Medical Missions CHA'
df_PCG_All.loc[df_PCG_All['EIN'] == '92-6001185', 'org_name'] = 'Crime Victim Compensation Fund'
df_PCG_All = df_PCG_All.groupby(['EIN','org_name'], as_index = False).sum()
#grouped so each EIN appears 1 time
#604 unique organizations by EIN (df_PCG_All['EIN'].nunique())

In [21]:
#new is 2007 - 2015
#old is 1998 - 2008
df_New_reg = df_New_All[['EIN', 'FiscalYearEnd', 'membership_dues', 
                                    'government_grants', 'service_revenue', 'fundraising']].fillna(0)
df_Old_reg = df_Old_All[['EIN', 'FiscalYearEnd', 'membership_dues', 
                                    'government_grants', 'service_revenue', 'fundraising']].fillna(0)

In [22]:
#invert PCG data
df_vert_PCG = pd.melt(df_PCG_All, id_vars=['EIN', 'org_name'], value_vars=['2009','2010','2011','2012','2013',
                                    '2014','2015','2016','2017','2018'], var_name = 'Year', value_name = 'Raised')

In [23]:
#assume year started is min year where raised > 0
df_PCG_NOZERO = df_vert_PCG[df_vert_PCG['Raised'] != 0]
df_PCG_Start = df_PCG_NOZERO.groupby('EIN', as_index=False)['Year'].min().rename(index=str, columns = {"Year": "PCGStartYear"})

In [24]:
#add the old and new together
df_reg = pd.concat([df_New_reg, df_Old_reg], ignore_index = True)

In [26]:
#adding TREATMENT dummy
df_reg['PCG_Treatment'] = df_reg['EIN'].isin(df_PCG_Start['EIN']).astype(int)
df_reg_PCGStart = df_reg.merge(df_PCG_Start, on = 'EIN', how='left').fillna(0)
#make sure int
df_reg_PCGStart['PCGStartYear'] = df_reg_PCGStart['PCGStartYear'].astype(int)

In [30]:
df_PCG_Raised = df_vert_PCG[['EIN', 'Year', 'Raised']].rename(index=str,columns={"Year" : "FiscalYearEnd"})
df_reg_PCGStart['FiscalYearEnd'] = df_reg_PCGStart['FiscalYearEnd'].astype(int)
df_PCG_Raised['FiscalYearEnd'] = df_PCG_Raised['FiscalYearEnd'].astype(int)
df_reg_Raised = pd.merge(df_reg_PCGStart,df_PCG_Raised, on = ['EIN', 'FiscalYearEnd'], how = 'left').fillna(0)

In [41]:
#add the POST_TREATMENT dummy
df_reg_Raised['PostTreatment'] = 0
#want to do anything about mid year ends???? below (some pcg donations may come after midyear fiscal end)
#if not in treatment - post_treatment is 1 if year after 2008
df_reg_Raised.loc[df_reg_Raised['PCG_Treatment'] == False, 'PostTreatment'] = \
                                    (df_reg_Raised['FiscalYearEnd'] > 2008).astype(int)
#if in treatment - post_treatment is 1 if year on/after PCGStartYear
df_reg_Raised.loc[df_reg_Raised['PCG_Treatment'] == True, 'PostTreatment'] = \
                                    (df_reg_Raised['FiscalYearEnd'] >= df_reg_Raised['PCGStartYear']).astype(int)
#NOW POSTTREATMENT IS CORRECT AND PCG_TREATMENT IS CORRECT

In [42]:
#add columns for depvar and interaction
df_reg_Raised['total_donations'] = df_reg_Raised['membership_dues'] + \
                                     df_reg_Raised['government_grants'] + df_reg_Raised['service_revenue']
df_reg_Raised['total_donations_NO_GG'] = df_reg_Raised['membership_dues'] + \
                                           df_reg_Raised['service_revenue']
df_reg_Raised['PostTreatment*PCG_Treatment'] = df_reg_Raised['PostTreatment'] * df_reg_Raised['PCG_Treatment']

In [43]:
df_reg_Raised.to_csv('InitialRegressionData.csv')

In [15]:
df_reg.dtypes

EIN                   object
FiscalYearEnd          int64
membership_dues      float64
government_grants    float64
service_revenue      float64
fundraising          float64
PCG_Treatment          int64
dtype: object

In [16]:
df_PCG_Start.dtypes

EIN             object
PCGStartYear    object
dtype: object